In [0]:
# -2
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# -1
!wget -O "./drive/My Drive/Colab Notebooks/SigNet/signatures.rar" "http://www.cedar.buffalo.edu/NIJ/data/signatures.rar"

--2019-06-29 17:50:53--  http://www.cedar.buffalo.edu/NIJ/data/signatures.rar
Resolving www.cedar.buffalo.edu (www.cedar.buffalo.edu)... 128.205.237.100
Connecting to www.cedar.buffalo.edu (www.cedar.buffalo.edu)|128.205.237.100|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cedar.buffalo.edu/NIJ/data/signatures.rar [following]
--2019-06-29 17:50:53--  https://cedar.buffalo.edu/NIJ/data/signatures.rar
Resolving cedar.buffalo.edu (cedar.buffalo.edu)... 128.205.237.100
Connecting to cedar.buffalo.edu (cedar.buffalo.edu)|128.205.237.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 253587033 (242M) [text/plain]
Saving to: ‘./drive/My Drive/Colab Notebooks/SigNet/signatures.rar’

./drive/My Drive/Co 100%[===================>] 241.84M  63.3MB/s    in 4.6s    

2019-06-29 17:50:58 (53.0 MB/s) - ‘./drive/My Drive/Colab Notebooks/SigNet/signatures.rar’ saved [253587033/253587033]



In [1]:
# 0
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
from natsort import natsorted, ns

def get_image_list():
  org_signs = os.listdir("drive/My Drive/Colab Notebooks/SigNet/signatures/signatures/full_org")
  forg_signs = os.listdir("drive/My Drive/Colab Notebooks/SigNet/signatures/signatures/full_forg")
  org_signs = [s for s in org_signs if s.endswith(".png")]
  forg_signs = [s for s in forg_signs if s.endswith(".png")]
  org_signs = natsorted(org_signs, alg=ns.IGNORECASE)
  forg_signs = natsorted(forg_signs, alg=ns.IGNORECASE)
  return org_signs, forg_signs

In [0]:
# data cleaning
def check_lists(org_signs,forg_signs):
  flag = False
  print(len(org_signs),len(forg_signs))
  for i in range(len(org_signs)):
    org_ext = org_signs[i][8:]
    forg_ext = forg_signs[i][9:]
    if org_ext != forg_ext:
      flag = True
      #print(i,org_ext,forg_ext)
  if(flag):
    print("Mismatches found")
  else:
    print("No mismatch found")

def refine_lists(org_signs,forg_signs):
  refined_org_signs = []
  for i in range(len(org_signs)):
    if "_41_" in org_signs[i]:
      continue
    refined_org_signs.append(org_signs[i])
  
  refined_forg_signs = []
  for i in range(len(forg_signs)):
    if "_41_" in forg_signs[i]:
      continue
    refined_forg_signs.append(forg_signs[i])
  return refined_org_signs, refined_forg_signs

def clean_lists(org_signs,forg_signs):
  org_signs, forg_signs = get_image_list()
  check_lists(org_signs,forg_signs)
  org_signs, forg_signs = refine_lists(org_signs,forg_signs)
  check_lists(org_signs,forg_signs)
  return org_signs, forg_signs

In [0]:
import random
import pandas as pd
import numpy as np

def get_dataframe(org_signs,forg_signs):
  no_of_ppl = len(org_signs)//24

  raw_data = {"image_1":[], "image_2":[], "label":[]}
  for i in range(no_of_ppl):
    i1_batch_1 = []
    i1_batch_2 = []
    i2_batch = []

    start = i*24
    end = (i+1)*24

    for j in range(start,end): 
      i1_batch_1.append(org_signs[j])
      i1_batch_2.append(org_signs[j])
      raw_data["label"].append(0)

    temp_rot = (i1_batch_1[-12:]+i1_batch_1[:-12])
    i1_batch_1.extend(i1_batch_2)

    for elem in temp_rot:
      i2_batch.append(elem)

    for j in range(start,end): 
      i2_batch.append(forg_signs[j])
      raw_data["label"].append(1)

    raw_data["image_1"].extend(i1_batch_1)
    raw_data["image_2"].extend(i2_batch)
  print(len(raw_data["image_1"]),len(raw_data["image_2"]),len(raw_data["label"]))
  df = pd.DataFrame(raw_data, columns = ["image_1","image_2","label"])
  df=df.reindex(np.random.permutation(df.index))
  return df

In [68]:
#generate labels
org_signs, forg_signs = get_image_list()
org_signs, forg_signs = clean_lists(org_signs, forg_signs)
df = get_dataframe(org_signs,forg_signs)

1316 1320
Mismatches found
1296 1296
No mismatch found
2592 2592 2592


In [69]:
df.head()

,image_1,image_2,label
2480,original_53_9.png,forgeries_53_9.png,1
1640,original_35_9.png,original_35_21.png,0
1608,original_34_1.png,forgeries_34_1.png,1
1379,original_29_12.png,forgeries_29_12.png,1
2473,original_53_2.png,forgeries_53_2.png,1


In [0]:
from sklearn.model_selection import train_test_split
train_set, val_set = train_test_split(df,test_size=0.3,random_state=0)

In [72]:
train_set.head()

,image_1,image_2,label
2498,original_54_3.png,original_54_15.png,0
1116,original_24_13.png,original_24_1.png,0
62,original_2_15.png,original_2_3.png,0
2326,original_50_23.png,original_50_11.png,0
2332,original_50_5.png,forgeries_50_5.png,1
